In [1]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

In [2]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [3]:
def get_data():
    (x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten = True, normalize = True, one_hot_label = False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
    return network

# 今回のneural networkは層少なめで行っている そのうち層の構造を厳密にして、一致率99%くらいを目指す
# なので、この荒いneural networkで動かしてみよう
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3) # オーバーフローしないように安全に正規化

    return y


In [14]:
x, t = get_data() # t: 10000個の数字が羅列されている 答え x: 手書きの数字のデータ
network = init_network()
accuracy_cnt = 0 # うまく特定できた回数の記録
for i in range(len(x)):
    # y: 0~9それぞれの確率が格納
    y = predict(network, x[i]) # i枚目の画像データをneural network にかける
    p = np.argmax(y) # 最も確率の高い要素のインデックスを取得
    if p == t[i]: # 答え合わせ
        accuracy_cnt += 1

print(accuracy_cnt) # 10000回中、9352回一致 
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))
print("len(x)",len(x))
print("len(t)",len(t))
print("len(y)",len(y))
print(t.shape)
print(x.shape)
print("答え",t)

9352
Accuracy:0.9352
len(x) 10000
len(t) 10000
len(y) 10
(10000,)
(10000, 784)
答え [7 2 1 ... 4 5 6]


In [5]:
x , _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']
print(x.shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)
# 重みと初期値の行列の次元が一致していることの確認e

(10000, 784)
(784, 50)
(50, 100)
(100, 10)


In [6]:
batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i + batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1)
    accuracy_cnt += np.sum(p == t[i:i + batch_size])
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352
